In [ ]:
#응답 csv 읽기 + 구조 확인
import pandas as pd
import os

# data/ 폴더에 응답 CSV 올려놓기 (이름 바꿔서)
df_resp = pd.read_csv("mp_survey/data/responses_raw.csv")

print("📋 응답 데이터 구조 확인:")
print("컬럼:", df_resp.columns.tolist())
print("\n샘플 데이터:")
print(df_resp.head(2))
print(f"\n👥 총 {len(df_resp)}명 응답자")

In [ ]:
# 앞에서 만든 메타데이터 로드
df_meta = pd.read_csv("mp_survey/meta/question_metadata.csv")
print("📋 문항 메타데이터 로드:", len(df_meta), "개 문항")

# 문항 번호 → 자아/테마 매핑 (빠른 조회용)
meta_by_no = df_meta.set_index("No")[["ego", "themes", "polarity"]].to_dict("index")
print("✅ 메타 매핑 준비 완료!")
print("\n샘플 매핑:")
for no in [1, 61, 65]:
    print(f"Q{no}: {meta_by_no[no]}")


In [ ]:
# 리커트 텍스트 → 숫자 점수
likert_score = {
    "전혀 그렇지 않다": 1,
    "그렇지 않다": 2,
    "보통이다": 3,
    "그렇다": 4,
    "매우 그렇다": 5
}

MAX_SCORE = 5

def text_to_score(answer_text, q_no):
    """응답 텍스트 → 점수 (polarity 반영)"""
    if pd.isna(answer_text):
        return None

    # 텍스트 정리
    ans = str(answer_text).strip()
    raw_score = likert_score.get(ans, None)
    if raw_score is None:
        return None

    # polarity 반영 (부정형 문항은 점수 뒤집기)
    meta = meta_by_no.get(q_no, {})
    polarity = meta.get("polarity", "positive")

    if polarity == "negative":
        return MAX_SCORE + 1 - raw_score  # 5→1, 4→2 리버스 스코어링
    return raw_score

print("✅ 점수 변환 함수 준비 완료!")


In [ ]:
# Q 컬럼만 골라내기
q_cols = [col for col in df_resp.columns if col.startswith("Q")]
print(f"🔍 발견된 문항 컬럼: {len(q_cols)}개 (Q1~Q{len(q_cols)})")

# Long 포맷으로 변환 (user × 문항 단위)
rows = []
for idx, row in df_resp.iterrows():
    user_id = row["이름"]  # 이름 컬럼을 user_id로 사용
    submitted_at = row["timestamp"]

    for q_col in q_cols:
        try:
            q_no = int(q_col[1:])  # Q61 → 61
            answer = row[q_col]
            score = text_to_score(answer, q_no)

            if score is not None:  # 유효한 응답만
                meta = meta_by_no.get(q_no, {})
                rows.append({
                    "user_id": user_id,
                    "submitted_at": submitted_at,
                    "q_no": q_no,
                    "answer": answer,
                    "score": score,
                    "ego": meta.get("ego", "미분류"),
                    "themes": meta.get("themes", "[]")
                })
        except:
            continue  # Q번호 파싱 실패시 스킵

# DataFrame으로 변환
df_long = pd.DataFrame(rows)
df_long.to_csv("mp_survey/processed/responses_long.csv", index=False, encoding="utf-8")

print(f"🎉 Long 포맷 변환 완료!")
print(f"📊 변환 결과: {len(df_long)}개 응답 → {df_long['user_id'].nunique()}명 사용자")
print("\n샘플 데이터:")
print(df_long.head())


In [ ]:
# 자아별 응답 분포 확인
print("📊 자아별 응답 분포:")
ego_dist = df_long['ego'].value_counts()
print(ego_dist)

print("\n👤 사용자별 응답 수:")
user_counts = df_long.groupby('user_id').size()
print(user_counts.describe())

print("\n📈 자아별 평균 점수 (탐색 정도):")
avg_scores = df_long.groupby(['user_id', 'ego'])['score'].mean().unstack(fill_value=0)
print(avg_scores.mean())

print("✅ 변환 검증 완료! 이제 사용자별 분석 가능!")
